# PARTIE A : IMPLEMENTATION DU JEU

In [1]:
import numpy as np
import pygame
import heapq

# Générer un puzzle initial aléatoire
def generate_puzzle(n):
    """Génère un puzzle aléatoire et vérifie qu'il est résoluble."""
    puzzle = np.arange(n * n)
    np.random.shuffle(puzzle)
    puzzle = puzzle.reshape((n, n))
    if not is_solvable(puzzle):
        # Si le puzzle n'est pas résoluble, échange deux cases non vides
        if puzzle[0, 0] != 0 and puzzle[0, 1] != 0:
            puzzle[0, 0], puzzle[0, 1] = puzzle[0, 1], puzzle[0, 0]
        else:
            puzzle[1, 0], puzzle[1, 1] = puzzle[1, 1], puzzle[1, 0]
    return puzzle

def is_solvable(puzzle):
    """Vérifie si un puzzle est résoluble."""
    flat_puzzle = puzzle.flatten()
    inversions = 0
    for i in range(len(flat_puzzle)):
        for j in range(i + 1, len(flat_puzzle)):
            if flat_puzzle[i] and flat_puzzle[j] and flat_puzzle[i] > flat_puzzle[j]:
                inversions += 1
    # Pour un puzzle NxN, si N est impair, le nombre d'inversions doit être pair.
    # Pour un N pair, il faut tenir compte de la position de la case vide.
    if len(puzzle) % 2 == 1:
        return inversions % 2 == 0
    else:
        empty_row = np.where(puzzle == 0)[0][0]
        return (inversions + empty_row) % 2 == 0
dim = 0
k = 0
while dim == 0 or dim > 5 or k == 0 :
    print("Enter la dimension (inférieur à 6): ", end="")  # end="" empêche le saut de ligne automatique
    # Lire l'entrée utilisateur (équivalent de scanf)
    dim = int(input())
    print("Choisir votre nombre de deplacement : ", end="")
    k = int(input())    

# Creer un matrice aleatoire
#numbers = np.random.choice(np.arange(1,101), size=dim**2, replace=False)

# Reshaper les nombres mélangés en une matrice 3x3
#matrix = numbers.reshape(dim, dim)
matrix = generate_puzzle(dim)

# Choisir une position aléatoire pour placer un 0
#row, col = np.random.randint(0, dim), np.random.randint(0, dim)
#matrix[row, col] = 0

# Recherche la position vide
p = np.where(matrix == 0)
lig_vide = p[0][0]
col_vide = p[1][0]

print("Matrice : \n", matrix)
print("Position vide : ({}, {})".format(lig_vide, col_vide))
print("k :", k)

pygame 2.5.2 (SDL 2.28.3, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
Enter la dimension (inférieur à 6): 

 4


Choisir votre nombre de deplacement : 

 3


Matrice : 
 [[10 11  3  1]
 [15 13  8  7]
 [12  4 14  5]
 [ 0  9  6  2]]
Position vide : (3, 0)
k : 3


# PARTIE B: Recherche des successeurs de l'element vide à base de l'Algorithme A*

In [3]:
def get_successeurs(noeud, matrix_size):    
    x, y = noeud
    successors = []
    
    # Déplacements possibles : 8 directions (haut, bas, gauche, droite, diagonales)
    mouvements = [
        (-1, 0), (1, 0),  # Haut, Bas
        (0, -1), (0, 1),  # Gauche, Droite        
    ]
    
    for dx, dy in mouvements:
        nx, ny = x + dx, y + dy
        
        # Vérifier que la nouvelle position est dans les limites de la matrice
        if 0 <= nx < matrix_size and 0 <= ny < matrix_size:
            successors.append((nx, ny))
    
    return successors

# Exemple d'utilisation
dim_matrix = dim  # Taille de la matrice 5x5
noeud_courant = (lig_vide, col_vide)  # Position actuelle
successeurs = get_successeurs(noeud_courant, dim_matrix)

print(f"Successeurs de {noeud_courant} : {successeurs}")

Successeurs de (3, 0) : [(2, 0), (3, 1)]


# EVENEMENT D'ECOUTE DE L'UTILISATEUR
# Partie jouable par l'utilisateur

In [6]:
# pygame setup
pygame.init()

screen = pygame.display.set_mode((1020, 620))
clock = pygame.time.Clock()
running = True

LARGEUR, HAUTEUR = 1020, 620

CELLULE_LARGEUR = LARGEUR // matrix.shape[1]
CELLULE_HAUTEUR = HAUTEUR // matrix.shape[0]

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
VERT = (0, 255, 0)  # Couleur rouge pour la cellule sélectionnée


# Changer la couleur de la cellule sélectionnée
cellule_colorier = (lig_vide, col_vide)
nouvelle_couleur = VERT

police = pygame.font.Font(None, 36)

while running:
    # poll for events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False    
        # Vérifier si une touche est enfoncée
        elif event.type == pygame.KEYDOWN:
            if (event.key == pygame.K_UP) and lig_vide > 0:    
                i1, j1 = lig_vide, col_vide 
                i2, j2 = lig_vide-1, col_vide
                matrix[i1, j1], matrix[i2, j2] = matrix[i2, j2], matrix[i1, j1]                
                lig_vide -= 1                   
            if (event.key == pygame.K_DOWN) and lig_vide < dim - 1:   
                i1, j1 = lig_vide, col_vide 
                i2, j2 = lig_vide+1, col_vide
                matrix[i1, j1], matrix[i2, j2] = matrix[i2, j2], matrix[i1, j1]
                lig_vide += 1                   
            if (event.key == pygame.K_LEFT) and col_vide > 0:   
                i1, j1 = lig_vide, col_vide 
                i2, j2 = lig_vide, col_vide-1
                matrix[i1, j1], matrix[i2, j2] = matrix[i2, j2], matrix[i1, j1]
                col_vide -= 1                   
            if (event.key == pygame.K_RIGHT) and col_vide < dim - 1:       
                i1, j1 = lig_vide, col_vide 
                i2, j2 = lig_vide, col_vide+1
                matrix[i1, j1], matrix[i2, j2] = matrix[i2, j2], matrix[i1, j1]
                col_vide += 1                   

            # Mettre à jour la cellule colorée
            cellule_colorier = (lig_vide, col_vide)

    # Remplissage de l'écran
    screen.fill(NOIR)

    # Dessin du tableau
    for i in range(matrix.shape[0]):  # Parcourt les lignes
        for j in range(matrix.shape[1]):  # Parcourt les colonnes
            # Coordonnées du coin supérieur gauche de la cellule
            x = j * CELLULE_LARGEUR
            y = i * CELLULE_HAUTEUR

            # Changer la couleur de la cellule sélectionnée
            if (i, j) == cellule_colorier:
                color = nouvelle_couleur
                text_color = VERT  # Texte en noir pour la cellule rouge
            else:
                color = NOIR
                text_color = BLANC  # Texte en noir pour toutes les autres cellules

            # Rectangle pour chaque cellule (fond de la cellule)
            pygame.draw.rect(screen, color, (x, y, CELLULE_LARGEUR, CELLULE_HAUTEUR))

            # Rectangle pour le contour des cellules (en blanc)
            pygame.draw.rect(screen, BLANC, (x, y, CELLULE_LARGEUR, CELLULE_HAUTEUR), 1)

            # Texte à afficher dans la cellule
            texte = police.render(str(matrix[i, j]), True, text_color)
            texte_rect = texte.get_rect(center=(x + CELLULE_LARGEUR // 2, y + CELLULE_HAUTEUR // 2))
            screen.blit(texte, texte_rect)

    # Mettre à jour l'affichage
    pygame.display.flip()

    # Limiter les FPS à 60
    clock.tick(60)

pygame.quit()

# A*

In [ ]:
pygame.init()

# Dimensions de l'écran
LARGEUR, HAUTEUR = 1020, 620
screen = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Résolution de puzzle avec A*")
clock = pygame.time.Clock()

# Couleurs
BLANC = (255, 255, 255)
NOIR = (0, 0, 0)
VERT = (0, 255, 0)
ROUGE = (255, 0, 0)

# Taille du puzzle
N = dim  # Taille du puzzle (3x3 pour un puzzle 8-tile)


# Changer la couleur de la cellule sélectionnée


# Police
police = pygame.font.Font(None, 72)

#dim = 0
#k = 0
""""""



# Calculer la distance de Manhattan
def manhattan_distance(puzzle):
    """Calcule la distance de Manhattan entre l'état actuel et l'état cible."""
    dist = 0
    for i in range(len(puzzle)):
        for j in range(len(puzzle)):
            value = puzzle[i][j]
            if value != 0:  # Ignorer la case vide
                target_x, target_y = divmod(value - 1, len(puzzle))
                dist += abs(target_x - i) + abs(target_y - j)
    return dist

# Implémentation de l'algorithme A*
def a_star(initial):
    """Résout le puzzle avec l'algorithme A*."""
    n = len(initial)    
    initial_flat = tuple(initial.flatten())
    goal = tuple(range(1, n * n)) + (0,)

    open_set = []
    heapq.heappush(open_set, (0, initial_flat, 0, None))
    visited = set()
    parents = {initial_flat: None}

    while open_set:
        _, current_state, g, _ = heapq.heappop(open_set)
        current_tuple = current_state

        if current_tuple in visited:
            continue
        visited.add(current_tuple)

        if current_tuple == goal:
            path = []
            while current_tuple:
                path.append(current_tuple)
                current_tuple = parents[current_tuple]
            return path[::-1]

        zero_index = current_state.index(0)
        zero_x, zero_y = divmod(zero_index, n)

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_x, new_y = zero_x + dx, zero_y + dy
            if 0 <= new_x < n and 0 <= new_y < n:
                new_state = list(current_state)
                swap_index = new_x * n + new_y
                new_state[zero_index], new_state[swap_index] = new_state[swap_index], new_state[zero_index]

                new_tuple = tuple(new_state)
                if new_tuple not in visited:
                    h = manhattan_distance(np.array(new_state).reshape((n, n)))
                    f = g + 1 + h
                    heapq.heappush(open_set, (f, new_tuple, g + 1, current_tuple))
                    parents[new_tuple] = current_tuple

    return None

# Initialiser le puzzle
#matrix = generate_puzzle(N)
solution_path = a_star(matrix)
current_step = 0

CELLULE_LARGEUR = LARGEUR // matrix.shape[1]
CELLULE_HAUTEUR = HAUTEUR // matrix.shape[0]  


# Boucle principale
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Afficher l'état actuel du puzzle
    screen.fill(NOIR)

    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            x = j * CELLULE_LARGEUR
            y = i * CELLULE_HAUTEUR

            value = matrix[i, j]
            color = BLANC if value != 0 else VERT
            text_color = NOIR if value != 0 else ROUGE

            pygame.draw.rect(screen, color, (x, y, CELLULE_LARGEUR, CELLULE_HAUTEUR))
            pygame.draw.rect(screen, NOIR, (x, y, CELLULE_LARGEUR, CELLULE_HAUTEUR), 2)

            if value != 0:
                text = police.render(str(value), True, text_color)
                text_rect = text.get_rect(center=(x + CELLULE_LARGEUR // 2, y + CELLULE_HAUTEUR // 2))
                screen.blit(text, text_rect)

    # Avancer dans le chemin solution
    if solution_path and current_step < len(solution_path):
        flat_state = solution_path[current_step]
        matrix = np.array(flat_state).reshape((N, N))
        current_step += 1

    pygame.display.flip()
    clock.tick(2)  # Limiter les mises à jour à 2 FPS pour observer les mouvements

pygame.quit()